In [1]:
from __future__ import absolute_import, division, print_function

import math
import os
import parser

import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier

import generar_jobs

In [2]:
# Parámetros hardcodeados del problema a estudiar
task_amount = 128
machine_amount = 4
task_heterogeneity = 0
machine_heterogeneity = 0
consistency_type = 0
accuracy_scores = []
classifiers = []

dimension = task_amount * machine_amount

# Referencia: https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
ns = 600  # Cantidad de instancias de entrenamiento
ni = dimension
no = 1  # Cantidad de salidas
alpha = 2
hidden_layer_amount = int(math.ceil(ns / (alpha * (ni + no))))
# Cada capa oculta tiene una cantidad de neuronas intermedia entre la cantidad
# de salidas y entradas
# Se pasa a una tupla para mandarlo al constructor del MLPClassifier
hidden_layer_neuron_amount = tuple([int(math.ceil((task_amount - no) / 2))]
                                   * hidden_layer_amount)

In [4]:
# Path base para todos los clasificadores
model_base_path = './models/' + str(task_amount) + 'x' + str(machine_amount) \
    + '-' + str(task_heterogeneity) + str(machine_heterogeneity) \
    + str(consistency_type) + '/'
baseDir = './ejemplos-entrenamiento-unificados/' + str(task_amount) + 'x' \
    + str(machine_amount) + '-' + str(task_heterogeneity) \
    + str(machine_heterogeneity) + str(consistency_type) + '/'
model_file_prefix = 'classifier'
model_file_extension = '.pkl'

for i in range(0, task_amount):
    # Se crea clasificador o se carga si ya existe
    try:
        classifier = joblib.load(model_base_path + model_file_prefix + str(i) \
                                 + model_file_extension)
    except Exception:
        print('El clasificador para la salida ' + str(i) + ' no existía.')
        classifier = MLPClassifier(solver='lbfgs', alpha=1e-5,
                                   hidden_layer_sizes=hidden_layer_neuron_amount,
                                   random_state=1)
    finally:
        # Se agrega a lista de clasificadores
        classifiers.append(classifier)

for i in range(0, task_amount):
    # Se cargan los datos
    TRAINING_FILE = baseDir + 'training/' + str(i) + '.csv'
    TEST_FILE = baseDir + 'test/' + str(i) + '.csv'
    training_set = pd.read_csv(TEST_FILE, header=None, delimiter=',')
    test_set = pd.read_csv(TRAINING_FILE, header=None, delimiter=',')
    # Se crea dataframe para datos y se separa el target
    df_training = pd.DataFrame(training_set)
    y_training = df_training.iloc[:, -1]
    # Se cargan datos de validacion
    df_test = pd.DataFrame(test_set)
    y_test = df_test.iloc[:, -1]
    # Se entrena clasificador con datos
    classifiers[i].fit(df_training.iloc[:, :-1], y_training)
    # Se genera el directorio del clasificador si no existe
    generar_jobs.generateDir(model_base_path)
    # Se persiste
    joblib.dump(classifiers[i], model_base_path + model_file_prefix + str(i) \
                + model_file_extension)
    # Determino accuracy para este clasificador
    results = []
    for j in range(0, len(df_test)):
        # Clasifico cada instancia del archivo de test, y agrego la clasificacion
        # a un array de resultados
        results.append(classifiers[i].predict(
            df_test.iloc[j][:-1].values.reshape(1, -1)))
    # Comparo los resultados con los esperados
    accuracy = accuracy_score(y_test, results)
    print("Precision de clasificador: " + str(accuracy))
    # Agrego accuracy a lista de accuracies
    accuracy_scores.append(accuracy)

Precision de clasificador: 0.513333333333
Precision de clasificador: 0.465
Precision de clasificador: 0.533333333333
Precision de clasificador: 0.478333333333
Precision de clasificador: 0.421666666667
Precision de clasificador: 0.518333333333
Precision de clasificador: 0.508333333333
Precision de clasificador: 0.536666666667
Precision de clasificador: 0.47
Precision de clasificador: 0.501666666667
Precision de clasificador: 0.501666666667
Precision de clasificador: 0.548333333333
Precision de clasificador: 0.515
Precision de clasificador: 0.493333333333
Precision de clasificador: 0.498333333333
Precision de clasificador: 0.496666666667
Precision de clasificador: 0.498333333333
Precision de clasificador: 0.516666666667
Precision de clasificador: 0.491666666667
Precision de clasificador: 0.486666666667
Precision de clasificador: 0.49
Precision de clasificador: 0.461666666667
Precision de clasificador: 0.48
Precision de clasificador: 0.475
Precision de clasificador: 0.443333333333
Precisi

In [5]:
# Calculo promedio de accuracy para todos los clasificadores
promedio = 0.
score_amount = len(accuracy_scores)
for i in range(0, score_amount):
    promedio += accuracy_scores[i]
promedio /= score_amount
print ('El promedio de accuracy es de {}'.format(promedio))

El promedio de accuracy es de 0.4948828125
